# Accessing data from a website
Not all websites make it easy to grab data. Luckily, `pandas` can help.

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

result = requests.get('https://en.wikipedia.org/wiki/List_of_sovereign_states')
pd.read_html(result.content)[0].head(20)

,Common and formal names,Membership within the UN System[c],Sovereignty dispute[d],Further information on status and recognition of sovereignty[f]
0,NaN,NaN,NaN,NaN
1,UN member states and General Assembly observer...,NaN,NaN,NaN
2,Abkhazia → See Abkhazia listing,Abkhazia → See Abkhazia listing,Abkhazia → See Abkhazia listing,Abkhazia → See Abkhazia listing
3,Afghanistan,UN member state,NaN,"The de facto ruling government, the Islamic Em..."
4,Albania – Republic of Albania,NaN,NaN,NaN
5,Algeria – People's Democratic Republic of Algeria,NaN,NaN,NaN
6,Andorra – Principality of Andorra,NaN,NaN,Andorra is a co-principality in which the offi...
7,Angola – Republic of Angola,NaN,NaN,NaN
8,Antigua and Barbuda,NaN,NaN,Antigua and Barbuda is a Commonwealth realm[g]...
9,Argentina – Argentine Republic[i],NaN,NaN,Argentina is a federation of 23 provinces and ...


For more complex parsing, we can utilize the `BeautifulSoup` library. Let's try to extract the same table, but use the new library. 

In [2]:
soup = BeautifulSoup(result.content, 'lxml') # Parse the HTML as a string
str(soup)[:500]

'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-language-alert-in-sidebar-enabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-enabled vector-feature-main-menu-pinned-disabled vector-feature-limited-width-enabled vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled vector-feature-page-tools-enabled" dir="l'

Find the tables.

In [3]:
tables = soup.find_all('table')

Using the `read_html` function of `pandas`, read the first table into a dataframe.

In [4]:
pd.read_html(str(tables[0]))[0].head(20)

,Common and formal names,Membership within the UN System[c],Sovereignty dispute[d],Further information on status and recognition of sovereignty[f]
0,NaN,NaN,NaN,NaN
1,UN member states and General Assembly observer...,NaN,NaN,NaN
2,Abkhazia → See Abkhazia listing,Abkhazia → See Abkhazia listing,Abkhazia → See Abkhazia listing,Abkhazia → See Abkhazia listing
3,Afghanistan,UN member state,NaN,"The de facto ruling government, the Islamic Em..."
4,Albania – Republic of Albania,NaN,NaN,NaN
5,Algeria – People's Democratic Republic of Algeria,NaN,NaN,NaN
6,Andorra – Principality of Andorra,NaN,NaN,Andorra is a co-principality in which the offi...
7,Angola – Republic of Angola,NaN,NaN,NaN
8,Antigua and Barbuda,NaN,NaN,Antigua and Barbuda is a Commonwealth realm[g]...
9,Argentina – Argentine Republic[i],NaN,NaN,Argentina is a federation of 23 provinces and ...


As we can see, the data we get back isn't always perfect, which is what's so nice about APIs instead of parsing HTML. Nevertheless, we would benefit a lot if we simplified this into a function.

In [5]:
def dfFromURL(url, tableNumber=1):
    soup = BeautifulSoup(requests.get(url).content, 'lxml') # Parse the HTML as a string
    tables = soup.find_all('table')
    # check table number is within number of tables on the page
    assert len(tables) >= tableNumber
    return pd.read_html(str(tables[tableNumber-1]))[0]

Now we can make a pretty simple call to get an HTML table as a dataframe. Let's try it.

In [6]:
prices = dfFromURL('https://finance.yahoo.com/quote/JPM/history?p=JPM')
prices.head()

,0
0,Will be right back... Thank you for your patie...


Got some messy data hear with divs and some disclaimers on the bottom...let's clean it up with a simple `dropna`.

In [7]:
prices = prices.dropna()
prices.head()

,0
0,Will be right back... Thank you for your patie...


Cool! Let's try to get the second table from a website. Let's see what the Cavs record was for the last few seasons:
    

In [8]:
df1 = dfFromURL('https://www.espn.com/nba/team/stats/_/name/cle', 0)
df1

,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,2PM,2PA,2P%,SC-EFF,SH-EFF
0,10.0,20.6,48.4,3.6,9.3,38.6,4.7,5.4,86.7,6.4,11.3,56.6,1.371,0.57
1,7.6,16.4,46.2,2.4,6.0,41.0,4.0,4.7,86.3,5.1,10.4,49.2,1.320,0.54
2,6.7,12.1,55.5,0.3,1.3,21.6,2.6,3.9,67.4,6.4,10.7,59.7,1.352,0.57
3,6.0,9.3,64.5,0.0,0.1,11.1,2.4,3.3,73.3,6.0,9.1,65.3,1.551,0.65
4,4.3,10.0,43.1,1.7,4.4,39.2,1.8,2.4,72.2,2.6,5.6,46.2,1.209,0.52
5,3.1,6.9,45.2,1.5,4.1,37.3,1.0,1.4,69.5,1.6,2.8,56.5,1.262,0.56
6,2.7,7.0,38.9,1.7,4.8,35.4,1.4,1.5,88.9,1.0,2.2,46.7,1.218,0.51
7,2.3,4.7,49.4,0.8,2.3,36.3,1.0,1.4,75.7,1.5,2.4,61.7,1.384,0.58
8,2.0,3.7,53.8,1.4,2.7,52.6,0.1,0.1,100.0,0.6,1.0,57.1,1.500,0.73
9,2.1,4.7,44.9,0.5,1.6,31.6,0.7,0.9,70.2,1.6,3.1,51.6,1.144,0.50


In [10]:
df2 = dfFromURL('https://www.espn.com/nba/team/stats/_/name/cle', 1)
df2

,Name
0,Donovan Mitchell SG
1,Darius Garland PG
2,Evan Mobley PF
3,Jarrett Allen C
4,Caris LeVert SG
5,Cedi Osman SF
6,Kevin Love PF *
7,Isaac Okoro SF
8,Danny Green F *
9,Lamar Stevens F


In [11]:
pd.concat((df2, df1), axis=1)

,Name,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,2PM,2PA,2P%,SC-EFF,SH-EFF
0,Donovan Mitchell SG,10.0,20.6,48.4,3.6,9.3,38.6,4.7,5.4,86.7,6.4,11.3,56.6,1.371,0.57
1,Darius Garland PG,7.6,16.4,46.2,2.4,6.0,41.0,4.0,4.7,86.3,5.1,10.4,49.2,1.320,0.54
2,Evan Mobley PF,6.7,12.1,55.5,0.3,1.3,21.6,2.6,3.9,67.4,6.4,10.7,59.7,1.352,0.57
3,Jarrett Allen C,6.0,9.3,64.5,0.0,0.1,11.1,2.4,3.3,73.3,6.0,9.1,65.3,1.551,0.65
4,Caris LeVert SG,4.3,10.0,43.1,1.7,4.4,39.2,1.8,2.4,72.2,2.6,5.6,46.2,1.209,0.52
5,Cedi Osman SF,3.1,6.9,45.2,1.5,4.1,37.3,1.0,1.4,69.5,1.6,2.8,56.5,1.262,0.56
6,Kevin Love PF *,2.7,7.0,38.9,1.7,4.8,35.4,1.4,1.5,88.9,1.0,2.2,46.7,1.218,0.51
7,Isaac Okoro SF,2.3,4.7,49.4,0.8,2.3,36.3,1.0,1.4,75.7,1.5,2.4,61.7,1.384,0.58
8,Danny Green F *,2.0,3.7,53.8,1.4,2.7,52.6,0.1,0.1,100.0,0.6,1.0,57.1,1.500,0.73
9,Lamar Stevens F,2.1,4.7,44.9,0.5,1.6,31.6,0.7,0.9,70.2,1.6,3.1,51.6,1.144,0.50


In [12]:
import datetime
print(datetime.datetime.now())

2023-04-09 01:44:54.171663


I have done reading this passage now. 2023-04-09 01:44:54.171663